In [ ]:
## TOTAL MEAN WIND - FROM MONTHLY DATA
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cartopy #Instead of from mpl_toolkits.basemap import Basemap "conda install -c conda-forge cartopy"
from netCDF4 import Dataset #"conda install -c anaconda netcdf4"
#from scipy.io import netcdf -- Alternative to netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr                                                    # import xarray library. Necessary for daily
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

#Download 10m wind speed, wind u and windv, 
#Wind speed sqrt(u^2 + v^2)

path = '.nc'#Path to ERA5 monthly wind speed and direction
data = Dataset(path)

lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
wind = data.variables['si10'][:,:,:] #wind
u = data.variables['u10'][:,:,:]
v = data.variables['v10'][:,:,:]


meanwind = wind.mean(axis=(0))
meanU = u.mean(axis=(0))
meanV = v.mean(axis=(0))




##PLOT


# Normalise the data for uniform arrow size
u_norm = 5*(meanU / np.sqrt(meanU ** 2.0 + meanV ** 2.0))
v_norm = 5*(meanV / np.sqrt(meanU ** 2.0 + meanV ** 2.0))

plt.figure(figsize=(10,10), dpi= 90)
ax = plt.axes(projection=ccrs.SouthPolarStereo(true_scale_latitude=-70))
theplot = plt.pcolormesh(lons,lats,meanwind,transform=ccrs.PlateCarree(),cmap = 'hot_r', vmin=6, vmax=10) #Set colorbar max/min
plt.quiver(lons, lats, u_norm, v_norm, pivot='middle', transform=ccrs.PlateCarree(), scale_units='inches', scale=40, color = 'blue')#pivot='middle'
ax.coastlines()
ax.set_extent([-122.5, -98, -74.5, -71], ccrs.PlateCarree())  

cbar = plt.colorbar(theplot, shrink=1)
cbar.set_label('Wind speed ms $^{-1}$',fontsize=14)




import geopandas as gpd
df = gpd.read_file('.shp') #Path to ASP study area shapefile
df.plot(ax=ax,color='none', edgecolor='lime',linewidth=3)


plt.savefig('.tif',dpi=300) #Path to save fig
